DEEP-FIRST SEARCH ALGORITHM IN GRAPHS

In [394]:
import pandas as pd
import numpy as np
from gdf_output import *

In [395]:
class Graph:
    def __init__(self, archive):
        self.nvertex, self.adjmatrix, self.adjlist = self.openFile(archive)

    def openFile(self, archive):
        with open(archive, 'r') as file:
            nvertex = int(file.readline())
            line = file.readlines()
            adjmatrix = [list(map(lambda x: int(x), l.split())) for l in line[0:]]
            adjlist = [[] for _ in range(nvertex)]

            for i in range(nvertex):
                for j in range(nvertex):
                    if adjmatrix[i][j] == 1:
                        adjlist[i].append(j+1)

        return nvertex, adjmatrix, adjlist

    def printMatrix(self):
        print("Adjacency Matrix:")
        print(" ", np.arange(1, len(self.adjmatrix)+1, 1))
        for i, l in enumerate(self.adjmatrix):
            print(f'{i+1} |{" ".join(map(str, l))}')

    def printList(self):
        print("\nAdjacency List:")
        for i, ladj in enumerate(self.adjlist):
            print(f'{i+1}: {"-> ".join(map(str, ladj))}')

In [396]:

class DeepSearch:
    def __init__(self, adjlist): #adjacent adjlist
        self.adjlist = adjlist
        self.result = self.startDFS()
        

    def inicializar(self):
        n = int(len(self.adjlist))
        time = 0
        output = {'in': [0] * n, 'out': [0] * n, 'parent': [0] * n}
        return time, n, output
    

    def DFS(self, v, time, output):
        edges = []
        time += 1
        output['in'][v] = time
        for w in self.adjlist[v]:
            w -= 1
            if output['in'][w] == 0:
                output['parent'][w] = v+1
                edges.append((v+1, w+1, '0,0,255'))  # azul
                time, output, edge = self.DFS(w, time, output)
                edges.extend(edge)
            if output['out'][w] == 0:
                if w+1 != output['parent'][v]:
                    print("Return edge") #return to visited vertex
                else:
                    print("Return edge") #return to parent vertex        
        time += 1
        output['out'][v] = time
        return time, output, edges
    

    def startDFS(self):
        time, n, output = self.inicializar()
        edges = []
        for i in range(n):
            if output['in'][i] == 0:
                time, result, edge = self.DFS(i, time, output)
                edges.extend(edge)
                
                
        #ORDENANDO AS ARESTAS DE FORMA LEXOGRAFICA
        for i, edge in enumerate(edges):
            if edge[0] > edge[1]:
                # Troca os elementos da aresta
                edges[i] = (edge[1], edge[0], edge[2])
        Edges_sorted = sorted(edges, key=lambda x: (x[0], x[1]))

        #JUNTANDO OS RESULTADOS
        result = {'n': n, 'vparent': output['parent'], 'vchild': range(1, n+1),
                  'in': output['in'], 'out': output['out'], 'Edges': Edges_sorted}


        return result


    def printDFS(self,output):
        df = pd.DataFrame({"Parent": output['vparent'],"Child": output['vchild'], 'Time in': output['in'], 'Time out': output['out']})

        # Ordenando o DataFrame
        dfSort = df.sort_values(by='Parent')
                
        print("\n RESULTADO: ----------------------------------------")
        print(dfSort)

  

In [397]:
#Lendo o grafo e imprimindo o grafo
archive = 'graph01.txt'
graph = Graph(archive)
graph.printMatrix()
graph.printList()

Adjacency Matrix:
  [ 1  2  3  4  5  6  7  8  9 10]
1 |0 0 1 0 0 0 0 0 0 0
2 |0 0 0 1 0 1 0 0 1 0
3 |1 0 0 0 0 1 0 0 0 1
4 |0 1 0 0 0 0 0 0 0 0
5 |0 0 0 0 0 1 1 0 0 0
6 |0 1 1 0 1 0 0 0 0 0
7 |0 0 0 0 1 0 0 0 0 0
8 |0 0 0 0 0 0 0 0 0 1
9 |0 1 0 0 0 0 0 0 0 0
10 |0 0 1 0 0 0 0 1 0 0

Adjacency List:
1: 3
2: 4-> 6-> 9
3: 1-> 6-> 10
4: 2
5: 6-> 7
6: 2-> 3-> 5
7: 5
8: 10
9: 2
10: 3-> 8


In [398]:
dfs =  DeepSearch(graph.adjlist)
# Chamando o método para iniciar a busca em profundidade
dfs_res = dfs.result
dfs.printDFS(dfs_res)

Return edge
Return edge
Return edge
Return edge
Return edge
Return edge
Return edge
Return edge
Return edge

 RESULTADO: ----------------------------------------
   Parent  Child  Time in  Time out
0       0      1        1        20
2       1      3        2        19
3       2      4        5         6
8       2      9        7         8
5       3      6        3        14
9       3     10       15        18
6       5      7       11        12
1       6      2        4         9
4       6      5       10        13
7      10      8       16        17


In [399]:
result = "graph01_dfs.gdf"
gerarGDF(dfs_res, result, 0)